In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
df = pd.read_csv("data/regimes.csv", parse_dates=["date"], dayfirst=True)
df.set_index('date', inplace=True)
dummies = pd.read_csv("data/dummies.csv", parse_dates=["date"], dayfirst=True)
dummies.set_index("date", inplace=True)
halving_dates = [
    pd.to_datetime("2012-11-28"),
    pd.to_datetime("2016-07-09"),
    pd.to_datetime("2020-05-11"),
    pd.to_datetime("2024-04-20")
]
df

,btc,ln_btc,btc_diff,spgsci,ln_spgsci,spgsci_diff,gepu,ln_gepu,gepu_diff,umcsent,ln_umcsent,umcsent_diff,abs_diff,gepu_Diff,vix,ln_vix,vix_diff,usd,ln_usd,usd_diff
date,,,,,,,,,,,,,,,,,,,,
2011-09-01,8.22,2.106570,-0.282193,590.9992,6.381815,-0.132435,190.366666,5.248952,-0.120573,60.8,4.107590,0.021613,1.3,21.623265,36.53,3.598134,0.041929,89.7253,4.496753,0.032015
2011-10-01,4.87,1.583094,-0.523476,647.9552,6.473822,0.092007,166.231788,5.113383,-0.135569,63.7,4.154185,0.046595,2.9,-24.134878,32.83,3.491343,-0.106791,90.6098,4.506562,0.009810
2011-11-01,3.33,1.202972,-0.380122,658.0235,6.489241,0.015419,197.928925,5.287908,0.174525,69.9,4.247066,0.092881,6.2,31.697137,31.94,3.463859,-0.027484,91.2649,4.513766,0.007204
2011-12-01,3.42,1.229641,0.026668,644.9144,6.469118,-0.020123,182.558037,5.207068,-0.080840,75.0,4.317488,0.070422,5.1,-15.370888,25.05,3.220874,-0.242985,92.2056,4.524021,0.010255
2012-01-01,5.00,1.609438,0.379797,660.6774,6.493266,0.024148,161.791967,5.086311,-0.120757,75.3,4.321480,0.003992,0.3,-20.766071,20.23,3.007167,-0.213707,91.6463,4.517937,-0.006084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-01,97291.00,11.485462,0.336524,549.6400,6.309264,0.025672,362.814025,5.893890,0.105509,71.7,4.272491,-0.031574,-2.3,36.329939,15.87,2.764431,-0.009407,127.8140,4.850576,0.010074
2025-01-01,94388.00,11.455169,-0.030293,561.9200,6.331359,0.022096,365.025208,5.899966,0.006076,64.7,4.169761,-0.102730,-7.0,2.211183,16.76,2.818995,0.054565,129.0413,4.860133,0.009556
2025-02-01,100638.00,11.519285,0.064116,553.5848,6.316415,-0.014945,396.779326,5.983380,0.083414,57.0,4.043051,-0.126710,-7.7,31.754118,16.97,2.831447,0.012452,128.0665,4.852550,-0.007583


In [2]:
def test_stationarity(series):
    print(f"=== Augmented Dickey-Fuller Test (ADF) ***{series.name}*** ===")
    adf_result = adfuller(series, autolag='AIC')
    print(f"ADF Statistic: {adf_result[0]:.4f}")
    print(f"p-value: {adf_result[1]:.4f}")
    print("=> Reject null hypothesis (non-stationary)?", "✅ Yes" if adf_result[1] < 0.05 else "❌ No")
    print()

    print(f"=== KPSS Test (Level Stationarity) ***{series.name}*** ===")
    kpss_result = kpss(series, regression='c', nlags='auto')
    print(f"KPSS Statistic: {kpss_result[0]:.4f}")
    print(f"p-value: {kpss_result[1]:.4f}")
    print("=> Reject null hypothesis (stationary)?", "✅ Yes" if kpss_result[1] < 0.05 else "❌ No")
    print("============================================================")

In [3]:
def test_granger_causality(df, col_x, col_y, maxlags=1):
    data = df[[col_x, col_y]].dropna()

    # Fit VAR model
    model = VAR(data)
    results = model.fit(maxlags=maxlags)

    # Test both directions
    x_causes_y = results.test_causality(col_y, [col_x], kind='f')
    print(x_causes_y)
    y_causes_x = results.test_causality(col_x, [col_y], kind='f')

    return {
        f"{col_x} → {col_y}": {
            "F-statistic": x_causes_y.test_statistic,
            "p-value": x_causes_y.pvalue
        },
        f"{col_y} → {col_x}": {
            "F-statistic": y_causes_x.test_statistic,
            "p-value": y_causes_x.pvalue
        }
    }

In [4]:
df_full = df.merge(dummies, left_index=True, right_index=True, how='inner')
df_full = df_full[["btc_diff"] + list(dummies.columns)].dropna()
merged_df = df.merge(df_full, left_index=True, right_index=True, how="inner")
merged_df

,btc,ln_btc,btc_diff_x,spgsci,ln_spgsci,spgsci_diff,gepu,ln_gepu,gepu_diff,umcsent,...,vix,ln_vix,vix_diff,usd,ln_usd,usd_diff,btc_diff_y,regime0,regime1,regime2
date,,,,,,,,,,,,,,,,,,,,,
2011-10-01,4.87,1.583094,-0.523476,647.9552,6.473822,0.092007,166.231788,5.113383,-0.135569,63.7,...,32.83,3.491343,-0.106791,90.6098,4.506562,0.009810,-0.523476,0,0,1
2011-11-01,3.33,1.202972,-0.380122,658.0235,6.489241,0.015419,197.928925,5.287908,0.174525,69.9,...,31.94,3.463859,-0.027484,91.2649,4.513766,0.007204,-0.380122,0,0,1
2011-12-01,3.42,1.229641,0.026668,644.9144,6.469118,-0.020123,182.558037,5.207068,-0.080840,75.0,...,25.05,3.220874,-0.242985,92.2056,4.524021,0.010255,0.026668,0,0,1
2012-01-01,5.00,1.609438,0.379797,660.6774,6.493266,0.024148,161.791967,5.086311,-0.120757,75.3,...,20.23,3.007167,-0.213707,91.6463,4.517937,-0.006084,0.379797,0,0,1
2012-02-01,5.88,1.771557,0.162119,703.4976,6.556064,0.062799,142.459776,4.959060,-0.127252,76.2,...,18.42,2.913437,-0.093730,89.9605,4.499371,-0.018566,0.162119,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-01,60807.00,11.015460,0.059230,534.2031,6.280776,0.002473,230.153036,5.438744,0.047086,71.8,...,19.96,2.993730,0.122428,123.8310,4.818918,0.013835,0.059230,0,1,0
2024-11-01,69490.00,11.148938,0.133478,535.7093,6.283592,0.002816,326.484086,5.788381,0.349637,74.0,...,16.02,2.773838,-0.219892,126.5329,4.840502,0.021585,0.133478,0,1,0
2024-12-01,97291.00,11.485462,0.336524,549.6400,6.309264,0.025672,362.814025,5.893890,0.105509,71.7,...,15.87,2.764431,-0.009407,127.8140,4.850576,0.010074,0.336524,0,1,0


In [5]:
test_granger_causality(merged_df, "gepu_diff", "regime2", maxlags=1)

<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: gepu_diff does not Granger-cause regime2: fail to reject at 5% significance level. Test statistic: 0.394, critical value: 3.871>, p-value: 0.531>


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


{'gepu_diff → regime2': {'F-statistic': np.float64(0.3942789846564326),
  'p-value': np.float64(0.5305151138171664)},
 'regime2 → gepu_diff': {'F-statistic': np.float64(1.7861866737463683),
  'p-value': np.float64(0.18235908597370268)}}

In [6]:
test_granger_causality(df, "gepu_diff", "btc_diff", maxlags=1)

<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: gepu_diff does not Granger-cause btc_diff: reject at 5% significance level. Test statistic: 5.264, critical value: 3.871>, p-value: 0.022>


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


{'gepu_diff → btc_diff': {'F-statistic': np.float64(5.264006725118017),
  'p-value': np.float64(0.022426350464634162)},
 'btc_diff → gepu_diff': {'F-statistic': np.float64(0.738232857417248),
  'p-value': np.float64(0.390878852413567)}}

In [7]:
test_granger_causality(df, "umcsent_diff", "btc_diff", maxlags=1)

<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: umcsent_diff does not Granger-cause btc_diff: fail to reject at 5% significance level. Test statistic: 2.904, critical value: 3.871>, p-value: 0.089>


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


{'umcsent_diff → btc_diff': {'F-statistic': np.float64(2.903916662223231),
  'p-value': np.float64(0.08934762276681306)},
 'btc_diff → umcsent_diff': {'F-statistic': np.float64(0.7623971414577426),
  'p-value': np.float64(0.3832423992833559)}}

In [8]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import f

# Prepare dataset
df_full = df.merge(dummies, left_index=True, right_index=True, how='inner')
df_full = df_full[["btc_diff"] + list(dummies.columns)].dropna()

# Restricted model: only constant (overall mean)
y = df_full["btc_diff"]
X_restricted = sm.add_constant(np.ones(len(y)))
model_r = sm.OLS(y, X_restricted).fit()

# Unrestricted model: constant + dummies (drop one dummy to avoid multicollinearity)
dummy_cols = list(dummies.columns)
dummy_cols.remove("regime2")  # e.g. drop regime_1 as base category
X_unrestricted = df_full[dummy_cols]
X_unrestricted = sm.add_constant(X_unrestricted)
model_u = sm.OLS(y, X_unrestricted).fit()

# F-test
rss_r = np.sum(model_r.resid ** 2)
rss_u = np.sum(model_u.resid ** 2)
n = len(y)
k = X_unrestricted.shape[1] - X_restricted.shape[1]   # number of restrictions (number of dummies)
f_stat = ((rss_r - rss_u) / k) / (rss_u / (n - X_unrestricted.shape[1]))
p_value = 1 - f.cdf(f_stat, k, n - X_unrestricted.shape[1])

print(f"Structural break (mean only) F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")


Structural break (mean only) F-statistic: 8.7118
p-value: 0.0003


In [9]:
df_full

,btc_diff,regime0,regime1,regime2
date,,,,
2011-10-01,-0.523476,0,0,1
2011-11-01,-0.380122,0,0,1
2011-12-01,0.026668,0,0,1
2012-01-01,0.379797,0,0,1
2012-02-01,0.162119,0,0,1
...,...,...,...,...
2024-10-01,0.059230,0,1,0
2024-11-01,0.133478,0,1,0
2024-12-01,0.336524,0,1,0


In [10]:
# Prepare data: merge macro variables and regime dummies
df_full = df.merge(dummies, left_index=True, right_index=True, how='inner')
df_full["spgsci_diff_lag"] = df_full["spgsci_diff"].shift(1)
df_full["vix_lag"] = df_full["vix"].shift(1)
df_full["usd_diff_lag"] = df_full["usd_diff"].shift(1)
df_full["gepu_diff_lag"] = df_full["gepu_diff"].shift(1)
# Define dependent variable (regime0) and independent variables (current macro values)
y = df_full["btc_diff"]
X = df_full[["spgsci_diff", "vix", "usd_diff", "gepu_diff", "spgsci_diff_lag", "vix_lag", "usd_diff_lag", "gepu_diff_lag"]]
X = sm.add_constant(X)  # add constant for intercept

# Fit logit model
logit_model = sm.OLS(y, X, missing='drop')
logit_result = logit_model.fit(disp=0, cov_type='HAC', cov_kwds={'maxlags': 4})

# Show summary
print(logit_result.summary())

                            OLS Regression Results                            
Dep. Variable:               btc_diff   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     1.734
Date:                Mon, 12 May 2025   Prob (F-statistic):             0.0948
Time:                        21:07:45   Log-Likelihood:                -1.7081
No. Observations:                 160   AIC:                             21.42
Df Residuals:                     151   BIC:                             49.09
Df Model:                           8                                         
Covariance Type:                  HAC                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2184      0.076     

In [11]:
X

,const,spgsci_diff,vix,usd_diff,gepu_diff,spgsci_diff_lag,vix_lag,usd_diff_lag,gepu_diff_lag
date,,,,,,,,,
2011-10-01,1.0,0.092007,32.83,0.009810,-0.135569,NaN,NaN,NaN,NaN
2011-11-01,1.0,0.015419,31.94,0.007204,0.174525,0.092007,32.83,0.009810,-0.135569
2011-12-01,1.0,-0.020123,25.05,0.010255,-0.080840,0.015419,31.94,0.007204,0.174525
2012-01-01,1.0,0.024148,20.23,-0.006084,-0.120757,-0.020123,25.05,0.010255,-0.080840
2012-02-01,1.0,0.062799,18.42,-0.018566,-0.127252,0.024148,20.23,-0.006084,-0.120757
...,...,...,...,...,...,...,...,...,...
2024-10-01,1.0,0.002473,19.96,0.013835,0.047086,-0.007229,17.66,-0.006127,0.049106
2024-11-01,1.0,0.002816,16.02,0.021585,0.349637,0.002473,19.96,0.013835,0.047086
2024-12-01,1.0,0.025672,15.87,0.010074,0.105509,0.002816,16.02,0.021585,0.349637


In [12]:
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, acorr_ljungbox
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
residuals = logit_result.resid
X_for_het = X.dropna()
bg_test = acorr_breusch_godfrey(logit_result, nlags=5)
bg_stat, bg_pval = bg_test[0], bg_test[1]
bp_stat, bp_pval, _, _ = het_breuschpagan(residuals, X_for_het)
white_stat, white_pval, _, _ = het_white(residuals, X_for_het)
print("bp", bp_stat, bp_pval)
print("white", white_stat, white_pval)

lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
lb_stat = lb_test.iloc[0, 0]
lb_pval = lb_test.iloc[0, 1]
print("lb", lb_stat, lb_pval)

bp 6.7388776770986425 0.5650500032720546
white 46.14876168874588 0.3835034441742694
lb 25.806795124307015 0.004008911550170311


In [13]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset

# Assume you have fitted an OLS model

# Apply RESET test (default = add squared fitted values)
reset_test = linear_reset(logit_result, power=2, use_f=True)
print(reset_test)

<F test: F=5.300115228219937, p=0.02269965641030894, df_denom=150, df_num=1>


In [14]:
import statsmodels.api as sm
from scipy.stats import chi2

# Define variable groups for joint testing
var_groups = {
    "SPGSCI": ["spgsci_diff", "spgsci_diff_lag"],
    "VIX": ["vix", "vix_lag"],
    "USD": ["usd_diff", "usd_diff_lag"],
    "GEPU": ["gepu_diff", "gepu_diff_lag"],
    "ALL": ["spgsci_diff", "spgsci_diff_lag",
            "vix", "vix_lag",
            "usd_diff", "usd_diff_lag",
            "gepu_diff", "gepu_diff_lag"]
}

# Dependent variable
y = df_full["regime2"]

# Full model variables
all_vars = ["spgsci_diff", "vix", "usd_diff", "gepu_diff",
            "spgsci_diff_lag", "vix_lag", "usd_diff_lag", "gepu_diff_lag"]

# Loop over tests
for name, drop_vars in var_groups.items():
    # Define restricted and full variable sets
    keep_vars = [var for var in all_vars if var not in drop_vars]
    
    # Create X matrices
    X_restricted = sm.add_constant(df_full[keep_vars])
    X_full = sm.add_constant(df_full[all_vars])
    
    # Fit restricted and full Probit models
    restricted_model = sm.Probit(y, X_restricted, missing='drop').fit(disp=0)
    full_model = sm.Probit(y, X_full, missing='drop').fit(disp=0)
    
    # Likelihood ratio test
    LR_stat = 2 * (full_model.llf - restricted_model.llf)
    df_diff = full_model.df_model - restricted_model.df_model
    p_value = chi2.sf(LR_stat, df_diff)

    # Print result
    print(f"\nLR test for {name}:")
    print(f"LR-statistic = {LR_stat:.4f}, df = {df_diff}, p-value = {p_value:.4f}")



LR test for SPGSCI:
LR-statistic = 6.5067, df = 2.0, p-value = 0.0386

LR test for VIX:
LR-statistic = 2.6354, df = 2.0, p-value = 0.2678

LR test for USD:
LR-statistic = 1.2281, df = 2.0, p-value = 0.5412

LR test for GEPU:
LR-statistic = 1.6638, df = 2.0, p-value = 0.4352

LR test for ALL:
LR-statistic = 22.0263, df = 8.0, p-value = 0.0049


In [15]:
import pandas as pd
import numpy as np

# Store results
f_results = []

var_groups = {
    "SPGSCI": ["spgsci_diff", "spgsci_diff_lag"],
    "VIX": ["vix", "vix_lag"],
    "USD": ["usd_diff", "usd_diff_lag"],
    "GEPU": ["gepu_diff", "gepu_diff_lag"],
    "ALL (all 8 vars)": ["spgsci_diff", "spgsci_diff_lag",
                          "vix", "vix_lag",
                          "usd_diff", "usd_diff_lag",
                          "gepu_diff", "gepu_diff_lag"]
}

for group_name, variables in var_groups.items():
    indices = [logit_result.model.exog_names.index(v) for v in variables]
    R = np.zeros((len(indices), len(logit_result.params)))
    for i, idx in enumerate(indices):
        R[i, idx] = 1
    test_result = logit_result.f_test(R)
    f_results.append({
        "Group": group_name,
        "F-statistic": round(float(test_result.fvalue), 3),
        "p-value": round(float(test_result.pvalue), 4)
    })

# Create DataFrame
f_test_df = pd.DataFrame(f_results)

# Show table
print(f_test_df)


              Group  F-statistic  p-value
0            SPGSCI        0.348   0.7065
1               VIX        2.376   0.0964
2               USD        1.321   0.2699
3              GEPU        1.979   0.1417
4  ALL (all 8 vars)        1.734   0.0948


In [16]:
latex_table = f_test_df.to_latex(index=False,
                                 caption="Joint Significance F-Tests for Macroeconomic Variables",
                                 label="tab:f_tests",
                                 float_format="%.3f",
                                 column_format="lcc")

print(latex_table)

\begin{table}
\caption{Joint Significance F-Tests for Macroeconomic Variables}
\label{tab:f_tests}
\begin{tabular}{lcc}
\toprule
Group & F-statistic & p-value \\
\midrule
SPGSCI & 0.348 & 0.707 \\
VIX & 2.376 & 0.096 \\
USD & 1.321 & 0.270 \\
GEPU & 1.979 & 0.142 \\
ALL (all 8 vars) & 1.734 & 0.095 \\
\bottomrule
\end{tabular}
\end{table}



In [17]:
df_full["vix"]

date
2011-10-01    32.83
2011-11-01    31.94
2011-12-01    25.05
2012-01-01    20.23
2012-02-01    18.42
              ...  
2024-10-01    19.96
2024-11-01    16.02
2024-12-01    15.87
2025-01-01    16.76
2025-02-01    16.97
Name: vix, Length: 161, dtype: float64

In [18]:
test_stationarity(df_full["gepu_diff"])

=== Augmented Dickey-Fuller Test (ADF) ***gepu_diff*** ===
ADF Statistic: -10.4739
p-value: 0.0000
=> Reject null hypothesis (non-stationary)? ✅ Yes

=== KPSS Test (Level Stationarity) ***gepu_diff*** ===
KPSS Statistic: 0.3052
p-value: 0.1000
=> Reject null hypothesis (stationary)? ❌ No


/var/folders/04/5gzpw57x2dz6l5fs4drprf_h0000gn/T/ipykernel_84779/4248190186.py:10: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_result = kpss(series, regression='c', nlags='auto')


In [19]:
test_granger_causality(df_full, "usd_diff_lag", "regime2", maxlags=1)

<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: usd_diff_lag does not Granger-cause regime2: fail to reject at 5% significance level. Test statistic: 0.909, critical value: 3.871>, p-value: 0.341>


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


{'usd_diff_lag → regime2': {'F-statistic': np.float64(0.9089850635485331),
  'p-value': np.float64(0.34112190494479255)},
 'regime2 → usd_diff_lag': {'F-statistic': np.float64(1.558008988681422),
  'p-value': np.float64(0.21289286526684045)}}

In [20]:
y = df["btc_diff"]
df["spgsci_diff_lag"] = df["spgsci_diff"].shift(1)
df["vix_lag"] = df["vix_diff"].shift(1)
df["vix_lag"] = df["vix"].shift(1)
df["usd_diff_lag"] = df["usd_diff"].shift(1)
df["gepu_diff_lag"] = df["gepu_diff"].shift(1)
X_lags = df[[
    "spgsci_diff", "spgsci_diff_lag", "vix", "gepu_diff_lag", "vix", "vix_lag", "usd_diff", "usd_diff_lag"
]]

# Add constant term
X_lags = sm.add_constant(X_lags)

# Run OLS regression
model_lags = sm.OLS(y, X_lags, missing='drop')
results = model_lags.fit(cov_type="HC3")

# Perform F-test: jointly test if all lagged coefficients are zero
hypotheses_spgci = 'spgsci_diff_lag = 0, spgsci_diff = 0'
f_test_spgci = results.f_test(hypotheses_spgci)

hypotheses_gepu = 'gepu_diff_lag = 0, gepu_diff = 0'
f_test_gepu = results.f_test(hypotheses_gepu)

hypotheses_vix = 'vix_lag = 0, vix = 0'
f_test_vix = results.f_test(hypotheses_vix)

hypotheses_usd = 'usd_diff_lag = 0, usd_diff = 0'
f_test_usd = results.f_test(hypotheses_usd)

# Output regression summary and F-test result
print(results.summary())
print(f_test_spgci)
print(f_test_gepu)
print(f_test_vix)
print(f_test_usd)

AssertionError: 